In [1]:
import torch
torch.set_default_tensor_type(torch.FloatTensor)
from torch.autograd import Variable
from torch.nn import functional as F
from torchmetrics import Accuracy, Recall, Precision, Specificity, ConfusionMatrix
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.metrics import confusion_matrix

from sklearn.metrics import precision_recall_curve, average_precision_score,roc_curve, auc, precision_score, recall_score, f1_score, confusion_matrix, accuracy_score

import numpy as np
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import matplotlib.pyplot as plt
from collections import Counter
import random
import math
import torch.optim as optim
from tabulate import tabulate
from ray import tune

from fractions import Fraction

In [2]:
# Create Tensors to hold input and outputs.
beta_1=np.append(np.array([4,4,-8,-4,6,8,-4,4,-6,-8]),np.zeros(90))
beta_2=np.append(np.array([-2,2,4,-2,3,-4,2,-2,-3,4]),np.zeros(90))
beta_3=np.append(np.array([-3,-3,6,-3,-4.5,-6,3,-3,4.5,6]),np.zeros(90))

beta_1=beta_1.reshape(-1,1)
beta_2=beta_2.reshape(-1,1)
beta_3=beta_3.reshape(-1,1)

n1=200
n2=200
n3=200
p=100
corval=0
prior=[11,12,13,14,15]

def sigmoid(X):
    return .5 * (1 + np.tanh(.5 * X))

def generate_data(corval,beta,n,p):
    mean=np.zeros(p)
    sigma=np.array([[corval**abs(i-j) for i in range(p)] for j in range(p)])
    x=torch.tensor(np.random.multivariate_normal(mean=mean,cov=sigma,size=n))
    Pi_test= sigmoid(x@beta)
    y=np.random.binomial(1,Pi_test.ravel(),n)
    x=x.to(torch.float32)
    #y=torch.tensor(y,dtype=torch.double)
    y=torch.tensor(y,dtype=torch.float)
    #y=y.to(torch.float32)
    return x,y
def setup_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.deterministic = True

def cubic_root(x):
    return math.copysign(math.pow(abs(x), 1.0/3.0), x)

In [3]:
def train_model(config):
    split_num=5
    kf = KFold(n_splits=split_num)



    sum_test_BCE_loss=0
    for idx_train,idx_test in kf.split(inputs):
        lambda1,lambda2,lr,ga= config["lambda1"], config["lambda2"],config["lr"],config["ga"]
        model=MLP(seed=1)
        max_iteration=1000
        learning_rate=lr
        params_to_optimize=list(model.parameters())
        optimizer = torch.optim.SGD(params_to_optimize, lr=learning_rate)
        loss_fn = torch.nn.BCELoss()
        scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[100,200,300,400,500,600,700,800], gamma=ga)

        BCE=[]

        for t in range(max_iteration):
            optimizer.zero_grad() # renew optimizer
            out, layer1_out, layer2_out= model(inputs[idx_train])

            # extract parameters
            #[:-1] for leaving out bias term#
            model_all_linear1_params = model.linear1
            model_all_linear2_params= torch.cat([x.view(-1) for x in model.linear2.parameters()][:-1])
            model_all_linear3_params= torch.cat([x.view(-1) for x in model.linear3.parameters()][:-1])

    


            # compute loss
            all_linear1_params=model_all_linear1_params.reshape(1,-1)
            all_linear23_params=torch.cat((model_all_linear2_params,model_all_linear3_params),0)
            
            non_prior=np.setdiff1d([i for i in range(all_linear1_params.shape[1])],prior)
            BCE_loss = loss_fn(out, targets[idx_train].reshape(-1,1))


#             l1_regularization = lambda1 * smooth_l1(all_linear1_params,prior)
            l1_regularization = lambda1 * torch.sum(torch.norm(all_linear1_params[:,non_prior],p=2,dim=0))

            l2_regularization = lambda2* torch.sum(all_linear23_params.pow(2))

            loss =BCE_loss+l1_regularization +l2_regularization

            # record loss
            BCE.append(BCE_loss.item())

            # compute derivative
            loss.backward()

            # gradient descent
            optimizer.step()

            # learning rate decay
            scheduler.step()
        
        
        test_out, test_layer1_out, test_layer2_out= model(train_x[idx_test])

        
        test_BCE_loss = loss_fn(test_out, targets[idx_test].reshape(-1,1))

        
        sum_test_BCE_loss+=test_BCE_loss.item()

    #print("loss: ",)
    tune.report(my_test_BCE_loss=sum_test_BCE_loss/split_num)

In [4]:
def train_model_2(config):
    split_num=5
    kf = KFold(n_splits=split_num)
    
    
    lambda1,lambda2,lr,eta = config["lambda1"], config["lambda2"],config["lr"],config["eta"]
    inputs=train_x
    targets=(1-eta)*train_y+eta*y_prior
    
    
    sum_test_BCE_loss=0
    for idx_train,idx_test in kf.split(inputs):
       
        
        model=MLP(seed=1)

        max_iteration=1000
        learning_rate=lr
        params_to_optimize=list(model.parameters())
        optimizer = torch.optim.SGD(params_to_optimize, lr=learning_rate)
        loss_fn = torch.nn.BCELoss()
        scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[100,200,300,400,500,600,700,800], gamma=0.3)

        BCE=[]

        for t in range(max_iteration):
            optimizer.zero_grad() # renew optimizer
            out, layer1_out, layer2_out,layer3_out= model(inputs[idx_train])


            # extract parameters
            #[:-1] for leaving out bias term#
            model_all_linear1_params = model.linear1
            model_all_linear2_params= torch.cat([x.view(-1) for x in model.linear2.parameters()][:-1])
            model_all_linear3_params= torch.cat([x.view(-1) for x in model.linear3.parameters()][:-1])
            model_all_linear4_params= torch.cat([x.view(-1) for x in model.linear4.parameters()][:-1])
    


            # compute loss
            all_linear1_params=model_all_linear1_params.reshape(1,-1)
            all_linear234_params=torch.cat((model_all_linear2_params,model_all_linear3_params, model_all_linear4_params),0)

            BCE_loss = loss_fn(out, targets[idx_train].reshape(-1,1))
            
            non_prior=np.setdiff1d([i for i in range(all_linear1_params.shape[1])],prior)
#             l1_regularization = lambda1 * smooth_l1(all_linear1_params,prior)
            l1_regularization = lambda1 * torch.sum(torch.norm(all_linear1_params[:,non_prior],p=2,dim=0))

            l2_regularization = lambda2* torch.sum(all_linear234_params.pow(2))
            loss =BCE_loss+l1_regularization +l2_regularization

            # record loss
            BCE.append(BCE_loss.item())

            # compute derivative
            loss.backward()

            # gradient descent
            optimizer.step()

            # learning rate decay
            scheduler.step()
        
        
        test_out, test_layer1_out, test_layer2_out,test_layer3_out= model(train_x[idx_test])

        test_BCE_loss = loss_fn(test_out, targets[idx_test].reshape(-1,1))
        
        sum_test_BCE_loss+=test_BCE_loss.item()


    #print("loss: ",)
    tune.report(my_test_BCE_loss=sum_test_BCE_loss/split_num)

In [5]:
TPR=[]
TNR=[]
GM=[]
accuracy=[]
precision=[]
f1=[]=[]
vsSEN=[]
vsSPE=[]
vsGM=[]
vsMR=[]
vsCCR=[]


vs = np.zeros((1,100))

In [6]:
for i in range(5):
    setup_seed(i)
       
    x1,y1=generate_data(corval,beta_1,n1,p)
    x2,y2=generate_data(corval,beta_2,n2,p)
    x3,y3=generate_data(corval,beta_3,n3,p)

    x= torch.cat((x1,x2), dim=0)
    x= torch.cat((x,x3), dim=0)

    y= torch.cat((y1,y2), dim=0)
    y= torch.cat((y,y3), dim=0)
    
    train_x, test_x, train_y, test_y= train_test_split(x,y, test_size=0.2, random_state=1)

    
    # switch training set
    inputs,targets = train_x,train_y

    # switching testing set
    test_inputs, test_targets = test_x,test_y
    
################## first step######################################  
    prior=[11,12,13,14,15]

# choose lambda1,lambda2 by ray tune
    class MLP(torch.nn.Module):
        def __init__(self,seed):
            super(MLP, self).__init__()
            torch.manual_seed(seed)
            self.linear1 = torch.nn.Parameter(torch.randn(p))
            self.linear2 = torch.nn.Linear(p,10)
            self.linear3 = torch.nn.Linear(10,1)

        def forward(self, x):
            layer1_out = self.linear1*x
            layer2_out = F.relu(self.linear2(layer1_out))
            out= torch.sigmoid(self.linear3(layer2_out))
            return out, layer1_out, layer2_out


    config={
            "lr":tune.choice([0.6,0.7,0.8]),
            "lambda1": tune.choice([0.03,0.04,0.05]),
            "lambda2": tune.choice([0.001,0.002,0.003]),
            "ga":tune.choice([0.3,0.4,0.5])
        }

    result = tune.run(
            train_model,  
            config=config,
            num_samples=30)
    
################## second step########################################    

    final_lambda=result.get_best_config('my_test_BCE_loss',mode='min')
    lambda1,lambda2,lr,ga=final_lambda["lambda1"],final_lambda["lambda2"],final_lambda["lr"],final_lambda["ga"]
    learning_rate=lr
    max_iteration=1000
    
    BCE=[]
    
    model=MLP(seed=1)

    loss_fn = torch.nn.BCELoss()
    params_to_optimize=list(model.parameters())
    optimizer = torch.optim.SGD(params_to_optimize, lr=learning_rate)
    
    scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[100,200,300,400,500,600,700,800], gamma=ga)

    loss_record=[]
    
    for t in range(max_iteration):
    
        # renew optimizer
        optimizer.zero_grad()

        # forward propagate
        out, layer1_out, layer2_out= model(inputs)



        # extract parameters
        #[:-1] for leaving out bias term#
        model_all_linear1_params = model.linear1
        model_all_linear2_params= torch.cat([x.view(-1) for x in model.linear2.parameters()][:-1])
        model_all_linear3_params= torch.cat([x.view(-1) for x in model.linear3.parameters()][:-1])

        # compute loss

        all_linear1_params=model_all_linear1_params.reshape(1,-1)
        all_linear23_params=torch.cat((model_all_linear2_params,model_all_linear3_params),0)

        BCE_loss = loss_fn(out, targets.reshape(-1,1))
        
        non_prior=np.setdiff1d([i for i in range(all_linear1_params.shape[1])],prior)
    #     l1_regularization = lambda1 * smooth_l1(all_linear1_params,prior)

        l1_regularization = lambda1 * torch.sum(torch.norm(all_linear1_params[:,non_prior],p=2,dim=0))

        l2_regularization = lambda2 * torch.sum(all_linear23_params.pow(2))

        loss =BCE_loss+l1_regularization +l2_regularization

        loss_record.append(loss.item())
        # record loss
        BCE.append(BCE_loss.item())

        # compute derivative
        loss.backward()

        # gradient descent
        optimizer.step()

        # learning rate decay
        scheduler.step()
    
################## Y_prior##################    
    y_prior= (model(train_x)[0]).detach().squeeze()

    
    
################## third step##################

    #no prior
    prior=[]
    # choose lambda1,lambda2 by ray tune
    class MLP(torch.nn.Module):
        def __init__(self,seed):
            super(MLP, self).__init__()
            torch.manual_seed(seed)
            self.linear1 = torch.nn.Parameter(torch.randn(p))
            self.linear2 = torch.nn.Linear(p,10)
            self.linear3 = torch.nn.Linear(10,10)
            self.linear4 = torch.nn.Linear(10,1)
        def forward(self, x):
            layer1_out = self.linear1*x
            layer2_out = F.relu(self.linear2(layer1_out))
            layer3_out = F.relu(self.linear3(layer2_out))
            out= torch.sigmoid(self.linear4(layer3_out))
            return out, layer1_out, layer2_out,layer3_out

    config={
        "lambda1": tune.choice([0.03,0.04,0.05]),
        "lambda2": tune.choice([0.001,0.002,0.003]),
         "eta": tune.choice([0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]),
         "lr":tune.choice([0.6,0.7,0.8])
                            }

    result = tune.run(
            train_model_2,  
            config=config,
            num_samples=150)
    

################## fourth step##################

    #no prior
    prior=[]
    #0.4,0.4
    # set hyperparamters
    final_parameter=result.get_best_config('my_test_BCE_loss',mode='min')
    lambda1,lambda2,eta,lr=final_parameter["lambda1"],final_parameter["lambda2"],final_parameter["eta"],final_parameter["lr"]
    learning_rate=lr
    max_iteration=1000

    inputs=train_x
    targets=(1-eta)*train_y+eta*y_prior

    # switching testing set
    test_inputs, test_targets = test_x,test_y

    # record loss descent
    BCE=[]


    # main nn object
    class MLP(torch.nn.Module):
        def __init__(self,seed):
            super(MLP, self).__init__()
            torch.manual_seed(seed)
            self.linear1 = torch.nn.Parameter(torch.randn(p))
            self.linear2 = torch.nn.Linear(p,10)
            self.linear3 = torch.nn.Linear(10,10)
            self.linear4 = torch.nn.Linear(10,1)

        def forward(self, x):
            layer1_out = self.linear1*x
            layer2_out = F.relu(self.linear2(layer1_out))
            layer3_out = F.relu(self.linear3(layer2_out))
            out= torch.sigmoid(self.linear4(layer3_out))
            return out, layer1_out, layer2_out,layer3_out



    model=MLP(seed=1)

    loss_fn = torch.nn.BCELoss()
    params_to_optimize=list(model.parameters())
    optimizer = torch.optim.SGD(params_to_optimize, lr=learning_rate)

    # learning rate decay scheme
    scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[100,200,300,400,500,600,700,800], gamma=0.3)

    loss_record=[]
    # loop for max_iteration times
    for t in range(max_iteration):

        # renew optimizer
        optimizer.zero_grad()

        # forward propagate
        out, layer1_out, layer2_out,layer3_out= model(inputs)



        model_all_linear1_params = model.linear1
        model_all_linear2_params= torch.cat([x.view(-1) for x in model.linear2.parameters()][:-1])
        model_all_linear3_params= torch.cat([x.view(-1) for x in model.linear3.parameters()][:-1])
        model_all_linear4_params= torch.cat([x.view(-1) for x in model.linear4.parameters()][:-1])

        # compute loss

        all_linear1_params=model_all_linear1_params.reshape(1,-1)
        all_linear234_params=torch.cat((model_all_linear2_params,model_all_linear3_params, model_all_linear4_params),0)

        BCE_loss = loss_fn(out, targets.reshape(-1,1))

        non_prior=np.setdiff1d([i for i in range(all_linear1_params.shape[1])],prior)

        l1_regularization = lambda1 * torch.sum(torch.norm(all_linear1_params[:,non_prior],p=2,dim=0))
        l2_regularization = lambda2 * torch.sum(all_linear234_params.pow(2))
        loss =BCE_loss+l1_regularization +l2_regularization

        loss_record.append(loss.item())
        # record loss
        BCE.append(BCE_loss.item())

        # compute derivative
        loss.backward()

        # gradient descent
        optimizer.step()

        # learning rate decay
        scheduler.step()
        
    best_trial = result.get_best_trial("my_test_BCE_loss", "min", "last")
    final_parameter=result.get_best_config('my_test_BCE_loss',mode='min')
    print(final_parameter)
    
####################################   prediction ####################################      

    prediction= (model(test_x)[0]>0.5).clone().int()
    target= test_y.reshape(-1,1).int()


    prediction=prediction.tolist()
    target=target.tolist()

    final_matrix=confusion_matrix(target,prediction)
    TPR.append(recall_score(target,prediction))#TPR
    TNR.append(final_matrix[0,0]/(sum(final_matrix[0,:])))#TNR
    GM.append(math.sqrt(TPR[-1]*TNR[-1]))
    accuracy.append(accuracy_score(target,prediction))
    precision.append(precision_score(target,prediction))
    f1.append(f1_score(target,prediction))

    
    
####################################   variable selection ####################################     
    model_weight=model_all_linear1_params.detach().numpy().copy().reshape(1,-1)

    threshold=0.12
    mcl_w=np.zeros(p)

    for j in range(p):
        mcl_w[j]=model_weight[0,j]

    max_mcl_w=max(abs(mcl_w))
    for j in range(p):
        if (abs(mcl_w[j]))<=threshold*max_mcl_w:
            model_weight[0,j]=0
        else:
            model_weight[0,j]=model_weight[0,j]
    
    index=np.nonzero(model_weight)[1]
    
    beta=np.array(beta_1)
    trueindex=np.where(beta!=0)[0]
    trueindex=trueindex.tolist()
    trueindex=set(trueindex)
    TP=len(trueindex.intersection(index))
    FP=len(np.setdiff1d(index,trueindex))
    FN=len(trueindex)-TP
    TN=p-len(trueindex)-FP
    
    vsSEN.append(TP/(TP+FN))
    vsSPE.append(TN/(TN+FP))
    vsGM.append(math.sqrt(vsSEN[-1]*vsSPE[-1]))
    vsMR.append((FP+FN)/(TP+FN+TN+FP))
    vsCCR.append(1-vsMR[-1])
    
    print(i)

2022-11-02 19:04:57,046	INFO worker.py:1509 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8274 
2022-11-02 19:04:57,948	WARNING function_trainable.py:619 -- Function checkpointing is disabled. This may result in unexpected behavior when using checkpointing features or certain schedulers. To enable, set the train function arguments to be `func(config, checkpoint_dir=None)`.


Trial name,status,loc,ga,lambda1,lambda2,lr,iter,total time (s),my_test_BCE_loss
train_model_3046f_00000,TERMINATED,10.224.0.12:23540,0.3,0.03,0.003,0.8,1,7.34647,0.723356
train_model_3046f_00001,TERMINATED,10.224.0.12:23566,0.5,0.03,0.001,0.8,1,7.70107,0.780572
train_model_3046f_00002,TERMINATED,10.224.0.12:23568,0.4,0.05,0.001,0.6,1,7.65784,0.707655
train_model_3046f_00003,TERMINATED,10.224.0.12:23570,0.4,0.05,0.002,0.8,1,7.23277,0.704883
train_model_3046f_00004,TERMINATED,10.224.0.12:23572,0.4,0.04,0.002,0.7,1,7.72247,0.709963
train_model_3046f_00005,TERMINATED,10.224.0.12:23574,0.4,0.03,0.002,0.8,1,7.26643,0.739351
train_model_3046f_00006,TERMINATED,10.224.0.12:23576,0.4,0.04,0.002,0.7,1,6.7733,0.709963
train_model_3046f_00007,TERMINATED,10.224.0.12:23578,0.4,0.03,0.001,0.7,1,7.10897,0.749035
train_model_3046f_00008,TERMINATED,10.224.0.12:23580,0.5,0.03,0.003,0.8,1,7.83234,0.727121
train_model_3046f_00009,TERMINATED,10.224.0.12:23582,0.5,0.03,0.001,0.6,1,7.75208,0.755726


(raylet) [2022-11-02 19:05:06,958 E 22613 22631] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-02_19-04-55_020890_22477 is over 95% full, available space: 372228096; capacity: 29394726912. Object creation will fail if spilling is required.


Result for train_model_3046f_00000:
  date: 2022-11-02_19-05-07
  done: false
  experiment_id: 2cd7f46eacfc478faa63b4c74c347cb0
  hostname: rd-PowerEdge-R940xa
  iterations_since_restore: 1
  my_test_BCE_loss: 0.7233561158180237
  node_ip: 10.224.0.12
  pid: 23540
  time_since_restore: 7.346465826034546
  time_this_iter_s: 7.346465826034546
  time_total_s: 7.346465826034546
  timestamp: 1667387107
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: 3046f_00000
  warmup_time: 0.0503535270690918
  
Result for train_model_3046f_00000:
  date: 2022-11-02_19-05-07
  done: true
  experiment_id: 2cd7f46eacfc478faa63b4c74c347cb0
  experiment_tag: 0_ga=0.3000,lambda1=0.0300,lambda2=0.0030,lr=0.8000
  hostname: rd-PowerEdge-R940xa
  iterations_since_restore: 1
  my_test_BCE_loss: 0.7233561158180237
  node_ip: 10.224.0.12
  pid: 23540
  time_since_restore: 7.346465826034546
  time_this_iter_s: 7.346465826034546
  time_total_s: 7.346465826034546
  timestamp: 1667387107
  timesteps_sin

Result for train_model_3046f_00005:
  date: 2022-11-02_19-05-11
  done: true
  experiment_id: b74e39e8768d4424becceb83d8a4f865
  experiment_tag: 5_ga=0.4000,lambda1=0.0300,lambda2=0.0020,lr=0.8000
  hostname: rd-PowerEdge-R940xa
  iterations_since_restore: 1
  my_test_BCE_loss: 0.7393511056900024
  node_ip: 10.224.0.12
  pid: 23574
  time_since_restore: 7.26643443107605
  time_this_iter_s: 7.26643443107605
  time_total_s: 7.26643443107605
  timestamp: 1667387111
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: 3046f_00005
  warmup_time: 0.005928516387939453
  
Result for train_model_3046f_00024:
  date: 2022-11-02_19-05-11
  done: false
  experiment_id: 5459f7b60ec24bf1beb96418a0aedf55
  hostname: rd-PowerEdge-R940xa
  iterations_since_restore: 1
  my_test_BCE_loss: 0.7648614168167114
  node_ip: 10.224.0.12
  pid: 23613
  time_since_restore: 7.204783916473389
  time_this_iter_s: 7.204783916473389
  time_total_s: 7.204783916473389
  timestamp: 1667387111
  timesteps_sinc

...

Result for train_model_2_a25ff_00144:
  date: 2022-11-02_19-16-28
  done: false
  experiment_id: 41a8dd2a5699483c9f62baf6a5cf2212
  hostname: rd-PowerEdge-R940xa
  iterations_since_restore: 1
  my_test_BCE_loss: 0.6984247446060181
  node_ip: 10.224.0.12
  pid: 5201
  time_since_restore: 10.067370176315308
  time_this_iter_s: 10.067370176315308
  time_total_s: 10.067370176315308
  timestamp: 1667387788
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: a25ff_00144
  warmup_time: 0.013214111328125
  
Result for train_model_2_a25ff_00144:
  date: 2022-11-02_19-16-28
  done: true
  experiment_id: 41a8dd2a5699483c9f62baf6a5cf2212
  experiment_tag: 144_eta=0.1000,lambda1=0.0300,lambda2=0.0020,lr=0.7000
  hostname: rd-PowerEdge-R940xa
  iterations_since_restore: 1
  my_test_BCE_loss: 0.6984247446060181
  node_ip: 10.224.0.12
  pid: 5201
  time_since_restore: 10.067370176315308
  time_this_iter_s: 10.067370176315308
  time_total_s: 10.067370176315308
  timestamp: 1667387788
  tim

2022-11-02 19:16:28,862	INFO tune.py:758 -- Total run time: 70.00 seconds (69.64 seconds for the tuning loop).


{'lambda1': 0.05, 'lambda2': 0.003, 'eta': 0.9, 'lr': 0.8}


/home/rd/anaconda3/envs/envJK/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


ValueError: math domain error

In [7]:
####################################   prediction ####################################   
from numpy import *
mean(TPR)
mean(TNR)
mean(GM)
mean(accuracy)
mean(precision)
mean(f1)


0.34903613335816724

0.7426415797317436

0.3404329085684977

0.5616666666666666

0.3457662337662338

0.34272723092134005

In [11]:
####################################   variable selection ####################################   
mean(vsSEN)
mean(vsSPE)
mean(vsGM)
mean(vsMR)
mean(vsCCR)

0.4

0.5844444444444445

0.2677871785461234

0.31

0.6900000000000001

In [9]:
vs

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.]])

In [10]:
i

4